# New Admin Created Quickly

**New user created and added to Administrators/Domain Admins within 30 minutes.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **who created account**, **where**, **ticket/HR correlation**, **new admin login activity**.


## Investigation Queries

Correlate user creation and group membership additions.


In [ ]:
// KQL
let window=30m;
let newUsers=SecurityEvent | where TimeGenerated>ago(30d) | where EventID==4720 | project CreatedTime=TimeGenerated, Target=TargetUserName, Creator=Account;
let addAdmin=SecurityEvent | where TimeGenerated>ago(30d) | where EventID in (4728,4732) | where TargetUserName in~ ('Domain Admins','Administrators') | project AddedTime=TimeGenerated, Member=MemberName, Group=TargetUserName, Actor=Account;
newUsers | join kind=inner (addAdmin) on $left.Target==$right.Member | where AddedTime-CreatedTime < window


In [ ]:
# Splunk SPL
index=auth sourcetype=WinEventLog:Security (EventCode=4720 OR EventCode=4728 OR EventCode=4732)
| stats earliest(_time) as first latest(_time) as last values(EventCode) as events values(Group_Name) as groups by Target_Account, Actor
| where mvcount(events)>=2


In [ ]:
# PowerShell
## Quick check of group membership (AD)
Get-ADGroupMember "Domain Admins" | Select-Object -ExpandProperty SamAccountName


## Containment & Response

Disable the new account and remove privileged group membership immediately if unauthorized.


In [ ]:
# PowerShell
Disable-ADAccount -Identity <NEW_ACCOUNT>
Remove-ADGroupMember -Identity "Domain Admins" -Members <NEW_ACCOUNT> -Confirm:$false


In [ ]:
# PowerShell
## Investigate creator account
# Revoke tokens / reset password if compromise suspected.
